# Create Web3 Smart Contract Agent with API Schema

## 1. Pre-requisites
Before starting, let's import the required packages and configure the support variables

In [ ]:
import logging
import boto3
import time
import zipfile
from io import BytesIO
import json
import uuid
import pprint
from knowledge_base import BedrockKnowledgeBase
from orchestration_management import OrchestrationManagement
from agent import create_agent_role_and_policies, create_lambda_role, delete_agent_roles_and_policies
from agent import create_dynamodb, create_lambda, clean_up_resources

In [ ]:
# setting logger
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [ ]:
# get boto3 clients for required AWS services
sts_client = boto3.client('sts')
iam_client = boto3.client('iam')
s3_client = boto3.client('s3')
lambda_client = boto3.client('lambda')
bedrock_agent_client = boto3.client('bedrock-agent')
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime')

In [ ]:
session = boto3.session.Session()
region = session.region_name
account_id = sts_client.get_caller_identity()["Account"]
region, account_id

In [ ]:
# Generate random prefix for unique IAM roles, agent name and S3 Bucket and 
# assign variables
suffix = f"{region}-{account_id}"
# Agent Configurition
agent_name = "web3-smart-contract-security-audit"
agent_alias_name = "security-audit-alias"
agent_description = "Web3 Smart Contract Security Audit"
code_audit_action_group_name = "CodeAuditActionGroup"
code_audit_action_group_description = "Actions for smart contract code audit"
knowledge_query_action_group_name = "KnowledgeQueryActionGroup"
knowledge_query_action_group_description = "Actions for smart contract related knowledge"
agent_instruction = """
You are an expert Web3 Smart Contract Security Auditor (SCSA) specialized in blockchain security, smart contract development, and vulnerability assessment.

Your primary functions are:
1. Answering questions about smart contract design principles, development best practices, and common vulnerabilities using structured knowledge base information.
2. Conducting comprehensive security audits of smart contract code to identify vulnerabilities and provide actionable recommendations.

For knowledge queries:
- Explain core concepts with practical examples.
- Highlight security considerations and best practices.

For code analysis:
- Evaluate structure and functionality.
- Identify vulnerabilities based on severity (Critical, High, Medium, Low).
- Provide clear recommendations for improvement.

Maintain a professional tone in all responses and ensure clarity, accuracy, and actionable guidance.

# You are an expert Web3 Smart Contract Security Auditor (SCSA) specialized in blockchain security, smart contract development, and vulnerability assessment. Your purpose is to assist users in understanding smart contract design principles and conducting comprehensive security audits.

# PRIMARY FUNCTIONS:

# 1. Knowledge Base Response Protocol:
# When users inquire about smart contract fundamentals:
# - Collect and analyze relevant information using core knowledge
# - Organize information in a structured format
# - Provide practical examples and real-world applications
# - Include security considerations and best practices

# 2. Code Analysis Protocol:
# When users present smart contract code:
# - Analyze code structure and functionality
# - Identify and classify security vulnerabilities
# - Evaluate against design principles and best practices
# - Provide detailed recommendations

# VULNERABILITY SEVERITY CLASSIFICATION:

# Critical Severity (Critical):
# - Immediate risk of fund loss
# - Direct theft of user funds possible
# - Complete contract compromise
# - Immediate action required

# High Severity (High):
# - Potential risk of fund loss
# - Core functionality compromised
# - Significant economic impact
# - Urgent action needed

# Medium Severity (Medium):
# - No immediate fund risk
# - Limited functionality impact
# - Moderate economic impact
# - Action recommended

# Low Severity (Low):
# - Minor optimization issues
# - Style guide violations
# - Best practice deviations
# - Optional improvements

# OUTPUT FORMATS:

# 1. For Knowledge Queries:
# [Topic Title]

# Core Concepts:

# Principal elements
# Key considerations
# Practical Implementation:

# Guidelines
# Best practices
# Example scenarios
# Security Considerations:

# Potential risks
# Prevention measures
# Security best practices
# Additional Resources:

# References
# Tools
# Further reading

# 2. For Code Analysis:
# [Security Audit Report]

# Executive Summary:

# Contract Purpose
# Audit Date
# Overall Risk Level
# Vulnerability Findings: Critical Findings:

# ID: [C-01]
# Title:
# Description:
# Impact:
# Recommendation:
# High Findings:

# ID: [H-01]
# Title:
# Description:
# Impact:
# Recommendation:
# Medium Findings:

# ID: [M-01]
# Title:
# Description:
# Impact:
# Recommendation:
# Low Findings:

# ID: [L-01]
# Title:
# Description:
# Impact:
# Recommendation:
# Detailed Analysis:

# Technical Details
# Affected Components
# Risk Explanation
# Fix Implementation
# Summary Recommendations:

# Critical Fixes
# Security Improvements
# Best Practices

# ANALYSIS METHODOLOGY:

# 1. Initial Assessment:
# - Understand contract purpose and scope
# - Identify key components and interactions
# - Review overall architecture
# - Analyze external dependencies

# 2. Detailed Review:
# - Conduct systematic code review
# - Pattern recognition for vulnerabilities
# - Security control evaluation
# - Gas optimization assessment
# - Classify vulnerabilities by severity
# - Evaluate potential impact

# 3. Recommendation Formation:
# - Prioritize findings by severity
# - Provide specific fix suggestions
# - Include preventive measures
# - Suggest security improvements

# QUALITY GUIDELINES:

# 1. For All Responses:
# - Maintain professional and educational tone
# - Provide clear, actionable recommendations
# - Include relevant examples when appropriate
# - Acknowledge limitations when present
# - Consider both direct and indirect impacts
# - Reference industry standards and best practices

# 2. For Knowledge Sharing:
# - Ensure accurate and up-to-date information
# - Provide clear explanation of concepts
# - Focus on practical applications
# - Maintain security-centric perspective

# 3. For Code Analysis:
# - Provide clear evidence for severity classifications
# - Include comprehensive vulnerability assessment
# - Explain technical details clearly
# - Offer implementation guidance
# - Reference similar known vulnerabilities when relevant

# Remember to:
# 1. Stay updated with latest security developments
# 2. Maintain accuracy and consistency in all responses
# 3. Provide practical, implementable solutions
# 4. Consider both security and usability
# 5. Be thorough in vulnerability assessment
# 6. Maintain consistent severity classifications
# 7. Acknowledge analysis limitations
# 8. Ask for clarification when needed
# """

# KB Configurition
knowledge_base_name = f'{agent_name}-kb'
knowledge_base_description = "Knowledge Base with smart contract design philosophies, development guidelines, and common vulnerabilities "

# S3 Bucket Configurition
kb_bucket_name = f'{agent_name}-kb-{suffix}'

schema_bucket_name = f'{agent_name}-schema-{suffix}'
schema_bucket_key = f'{agent_name}-schema.json'
# OpenAI Schemas Configuration
# TODO 需要更改 API
schema_name = 'web3-smart-contract-security-audit_agent_openapi_schema.json'
schema_arn = f'arn:aws:s3:::{openapi_bucket_name}/{openapi_bucket_key}'

# IAM Policy Configuration
bedrock_agent_bedrock_allow_policy_name = f"{agent_name}-allow-{suffix}"
bedrock_agent_s3_allow_policy_name = f"{agent_name}-s3-allow-{suffix}"
lambda_role_name = f'{agent_name}-lambda-role-{suffix}'
agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{suffix}'
lambda_code_path = "lambda_function.py"
lambda_name = f'{agent_name}-{suffix}'

# Amazon Bedrock FMs id Configuration
model_id = "anthropic.claude-3-5-sonnet-20241022-v2:0"
inference_model_id = "us.anthropic.claude-3-5-sonnet-20241022-v2:0"
embedding_model_id = "amazon.titan-embed-text-v2:0"

In [ ]:
## 2. Upload the openapi schema to Amazon S3

In [ ]:
# Create S3 bucket for Open API schema
if region == "us-east-1":
    s3bucket = s3_client.create_bucket(
        Bucket=schema_bucket_name
    )
else:
    s3bucket = s3_client.create_bucket(
        Bucket=schema_bucket_name,
        CreateBucketConfiguration={ 'LocationConstraint': region } 
    )

In [ ]:
# Upload Open API schema to this s3 bucket
s3_client.upload_file(schema_name, schema_bucket_name, schema_bucket_key)

## 3. Create Knowledge Base for Amazon Bedrock
Let's start by creating a Knowledge Base for Amazon Bedrock to store knowledge related to Web3 Smart Contract. Knowledge Bases allow you to integrate with different vector databases including Amazon OpenSearch Serverless, Amazon Aurora and Pinecone. For this example, we will integrate the knowledge base with Amazon OpenSearch Serverless. To do so, we will use the helper class BedrockKnowledgeBase which will create the knowledge base and all of its pre-requisites:

1. IAM roles and policies
2. S3 bucket
3. Amazon OpenSearch Serverless encryption, network and data access policies
4. Amazon OpenSearch Serverless collection
5. Amazon OpenSearch Serverless vector index
6. Knowledge base
7. Knowledge base data source

knowledge_base = BedrockKnowledgeBase(
    kb_name=knowledge_base_name,
    kb_description=knowledge_base_description,
    embedding_model_id = embedding_model_id,
    data_bucket_name=kb_bucket_name,
    schema_bucket_name = schema_bucket_name
)

In [ ]:
## 4. Upload the dataset to Amazon S3
Now that we have created the knowledge base, let's populate it with the menu's dataset. The Knowledge Base data source expects the data to be available on the S3 bucket connected to it and changes on the data can be syncronized to the knowledge base using the StartIngestionJob API call. In this example we will use the boto3 abstraction of the API, via our helper classe.

Let's first upload the Smart Contract related data available on the dataset folder to s3

In [ ]:
def upload_directory(path, bucket_name):
        for root,dirs,files in os.walk(path):
            for file in files:
                file_to_upload = os.path.join(root,file)
                print(f"uploading file {file_to_upload} to {bucket_name}")
                s3_client.upload_file(file_to_upload,bucket_name,file)

upload_directory("dataset", bucket_name)

In [ ]:
Now we start the ingestion job

In [ ]:
# ensure that the kb is available
time.sleep(30)
# sync knowledge base
knowledge_base.start_ingestion_job()

In [ ]:
Finally we collect the Knowledge Base Id to integrate it with our Agent later on

In [ ]:
kb_id = knowledge_base.get_knowledge_base_id()

In [ ]:
### 4.1 Test the Knowledge Base
Now the Knowlegde Base is available we can test it out using the [**retrieve**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve.html) and [**retrieve_and_generate**](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-agent-runtime/client/retrieve_and_generate.html) functions. 

#### Testing Knowledge Base with Retrieve and Generate API

Let's first test the knowledge base using the retrieve and generate API. With this API, Bedrock takes care of retrieving the necessary references from the knowledge base and generating the final answer using a LLM model from Bedrock

In [ ]:
# TODO need change modelArn 
response = bedrock_agent_runtime_client.retrieve_and_generate(
    input={
        "text": "What are the security best practices?"
    },
    retrieveAndGenerateConfiguration={
        "type": "KNOWLEDGE_BASE",
        "knowledgeBaseConfiguration": {
            'knowledgeBaseId': kb_id,
            "modelArn": "arn:aws:bedrock:{}::foundation-model/{}".format(region, agent_foundation_model),
            "retrievalConfiguration": {
                "vectorSearchConfiguration": {
                    "numberOfResults":5
                } 
            }
        }
    }
)

print(response['output']['text'],end='\n'*2)

In [ ]:
As you can see, with the retrieve and generate API we get the final response directly and we don't see the different sources used to generate this response. Let's now retrieve the source information from the knowledge base with the retrieve API.

In [ ]:
#### Testing Knowledge Base with Retrieve API

In [ ]:
If you need an extra layer of control, you can retrieve the chuncks that best match your query using the retrieve API. In this setup, we can configure the desired number of results and control the final answer with your own application logic. The API then provides you with the matching content, its S3 location, the similarity score and the chunk metadata

In [ ]:
response_ret = bedrock_agent_runtime_client.retrieve(
    knowledgeBaseId=kb_id, 
    nextToken='string',
    retrievalConfiguration={
        "vectorSearchConfiguration": {
            "numberOfResults":5,
        } 
    },
    retrievalQuery={
        'text': 'What are the security best practices?'
    }
)

def response_print(retrieve_resp):
#structure 'retrievalResults': list of contents. Each list has content, location, score, metadata
    for num,chunk in enumerate(response_ret['retrievalResults'],1):
        print(f'Chunk {num}: ',chunk['content']['text'],end='\n'*2)
        print(f'Chunk {num} Location: ',chunk['location'],end='\n'*2)
        print(f'Chunk {num} Score: ',chunk['score'],end='\n'*2)
        print(f'Chunk {num} Metadata: ',chunk['metadata'],end='\n'*2)

response_print(response_ret)

In [ ]:
## 5. Create Lambda funtion for Action Group

In [ ]:
Let's now create the lambda function required by the agent action group. We first need to create the lambda IAM role and it's policy. After that, we package the lambda function into a ZIP format to create the function

In [ ]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "bedrock:InvokeModel",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_role_name,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_role_name)

iam_client.attach_role_policy(
    RoleName=lambda_role_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

In [ ]:
Take a look at the Lambda function code that will be used as an Action group for the agent

In [ ]:
!pygmentize lambda_function.py

In [ ]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write(lambda_code_path)
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

## Create Lambda function for Action Group
Let's now create the lambda function required by the agent action group. We first need to create the lambda IAM role and it's policy. After that, we package the lambda function into a ZIP format to create the function

In [ ]:
# Create IAM Role for the Lambda function
try:
    assume_role_policy_document = {
        "Version": "2012-10-17",
        "Statement": [
            {
                "Effect": "Allow",
                "Action": "bedrock:InvokeModel",
                "Principal": {
                    "Service": "lambda.amazonaws.com"
                },
                "Action": "sts:AssumeRole"
            }
        ]
    }

    assume_role_policy_document_json = json.dumps(assume_role_policy_document)

    lambda_iam_role = iam_client.create_role(
        RoleName=lambda_role_name,
        AssumeRolePolicyDocument=assume_role_policy_document_json
    )

    # Pause to make sure role is created
    time.sleep(10)
except:
    lambda_iam_role = iam_client.get_role(RoleName=lambda_role_name)

iam_client.attach_role_policy(
    RoleName=lambda_role_name,
    PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole'
)

Take a look at the Lambda function code that will be used as an Action group for the agent

In [ ]:
# TODO Lambda 
!pygmentize lambda_function.py

In [ ]:
# Package up the lambda function code
s = BytesIO()
z = zipfile.ZipFile(s, 'w')
z.write(lambda_code_path)
z.close()
zip_content = s.getvalue()

# Create Lambda Function
lambda_function = lambda_client.create_function(
    FunctionName=lambda_name,
    Runtime='python3.12',
    Timeout=180,
    Role=lambda_iam_role['Role']['Arn'],
    Code={'ZipFile': zip_content},
    Handler='lambda_function.lambda_handler'
)

## Create Agent
We will now create the agent. To do so, we first need to create the agent policies that allow bedrock model invocation and the agent IAM role with the policy associated to it. We will allow this agent to invoke the Claude Sonnet model.

In [ ]:
# Create IAM policies for agent

bedrock_agent_bedrock_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AmazonBedrockAgentBedrockFoundationModelPolicy",
            "Effect": "Allow",
            "Action": "bedrock:InvokeModel",
            "Resource": [
                f"arn:aws:bedrock:{region}::foundation-model/{model_id}"
            ]
        }
    ]
}

bedrock_policy_json = json.dumps(bedrock_agent_bedrock_allow_policy_statement)

agent_bedrock_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_bedrock_allow_policy_name,
    PolicyDocument=bedrock_policy_json
)



Next, we will create a policy document that allows fetching of the Agent's OpenAPI schema from S3:

In [ ]:
bedrock_agent_s3_allow_policy_statement = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "AllowAgentAccessOpenAPISchema",
            "Effect": "Allow",
            "Action": ["s3:GetObject"],
            "Resource": [
                schema_arn
            ]
        }
    ]
}


bedrock_agent_s3_json = json.dumps(bedrock_agent_s3_allow_policy_statement)
agent_s3_schema_policy = iam_client.create_policy(
    PolicyName=bedrock_agent_s3_allow_policy_name,
    Description=f"Policy to allow invoke Lambda that was provisioned for it.",
    PolicyDocument=bedrock_agent_s3_json
)

Finally, create a role with the above two policies attached

## Creating Agent
Once the needed IAM role is created, we can use the bedrock agent client to create a new agent. To do so we use the `create_agent` function. It requires an agent name, underlying foundation model and instructions. You can also provide an agent description. Note that the agent created is not yet prepared. We will focus on preparing the agent and then using it to invoke actions and use other APIs

In [ ]:
# Create Agent
# TODO 修改 instruction
agent_instruction = """
You are an agent that can handle various tasks related to insurance claims, including looking up claim 
details, finding what paperwork is outstanding, and sending reminders. Only send reminders if you have been 
explicitly requested to do so. If an user asks about your functionality, provide guidance in natural language 
and do not include function names on the output."""

response = bedrock_agent_client.create_agent(
    agentName=agent_name,
    agentResourceRoleArn=agent_role['Role']['Arn'],
    description="Agent for smart contract security audit.",
    idleSessionTTLInSeconds=1800,
    foundationModel=model_id,
    instruction=agent_instruction,
)

Looking at the created agent, we can see its status and agent id

In [ ]:
response

Let's now store the agent id in a local variable to use it on the next steps

In [ ]:
agent_id = response['agent']['agentId']
agent_id

## Create Agent Action Group
We will now create and agent action group that uses the lambda function and API schema files created before.
The `create_agent_action_group` function provides this functionality. We will use `DRAFT` as the agent version since we haven't yet create an agent version or alias. To inform the agent about the action group functionalities, we will provide an action group description containing the functionalities of the action group.

In [ ]:
# Pause to make sure agent is created
time.sleep(30)
# Now, we can configure and create an action group here:
agent_action_group_response = bedrock_agent_client.create_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    actionGroupName='SecurityAuditActionGroup',
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': bucket_key
        }
    },
    description='Actions for Smart Contract Development Guide, Classic Vulnerabilities, Code Audit'
)

In [ ]:
agent_action_group_response

## Allowing Agent to invoke Action Group Lambda
Before using our action group, we need to allow our agent to invoke the lambda function associated to the action group. This is done via resource-based policy. Let's add the resource-based policy to the lambda function created

In [ ]:
# Create allow invoke permission on lambda
response = lambda_client.add_permission(
    FunctionName=lambda_name,
    StatementId='allow_bedrock',
    Action='lambda:InvokeFunction',
    Principal='bedrock.amazonaws.com',
    SourceArn=f"arn:aws:bedrock:{region}:{account_id}:agent/{agent_id}",
)

## Preparing Agent
Let's create a DRAFT version of the agent that can be used for internal testing.

In [ ]:
agent_prepare = bedrock_agent_client.prepare_agent(agentId=agent_id)
agent_prepare

## Create Agent alias
We will now create an alias of the agent that can be used to deploy the agent.

In [ ]:
# Pause to make sure agent is prepared
time.sleep(30)
agent_alias = bedrock_agent_client.create_agent_alias(
    agentId=agent_id,
    agentAliasName=agent_alias_name
)

In [ ]:
agent_alias

## Create a simple function to Invoke the Agent

In [ ]:
def simple_agent_invoke(input_text, agent_id, agent_alias_id, session_id=None, enable_trace=False, end_session=False):
    
    agentResponse = bedrock_agent_runtime_client.invoke_agent(
        inputText=input_text,
        agentId=agent_id,
        agentAliasId=agent_alias_id, 
        sessionId=session_id,
        enableTrace=enable_trace, 
        endSession= end_session
    )
    
    event_stream = agentResponse['completion']
    try:
        for event in event_stream:  
            if 'returnControl' in event:
                pprint.pp(event)
                
                return event
            elif 'chunk' in event:
                data = event['chunk']['bytes']
                logger.info(f"Final answer ->\n{data.decode('utf8')}")
                agent_answer = data.decode('utf8')
                end_event_received = True
                
                print(agent_answer)
                # End event indicates that the request finished successfully
                return event
            elif 'trace' in event:
                logger.info(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.", e)

## Extract the agentAliasId from the response

In [ ]:
time.sleep(30)

agent_alias_id = agent_alias['agentAlias']['agentAliasId']

## Invoke the Agent for an API that does not have User Confirmation Enabled

As the first test, we will invoke our agent with a prompt that we know is for an API where user confirmation has not been enabled. Now that we've created the agent, let's use the `bedrock-agent-runtime` client to invoke this agent and perform some tasks.

In [ ]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False

simple_agent_invoke("what are the open claims?", agent_id, agent_alias_id, session_id)

## Invoke the Agent for an API that has User Confirmation Enabled

In [ ]:
## create a random id for session initiator id
session_id:str = str(uuid.uuid1())
enable_trace:bool = False
end_session:bool = False

event = simple_agent_invoke("Please send a reminder to the insured about claim claim-006 only. Please reply indicating whether the reminder was sent or not", agent_id, agent_alias_id, session_id)

## Invoking the Agent with the function results and Allowed execution

Finally, we need to invoke the agent passing the function results as a parameter. This lets us use the agent for generating the final response. Note that if user confirmation is enabled for the function or API, the user will be presented with the option to `CONFIRM` or `DENY` the action specified in the response, within the `confirmationState` field. For this example, we will use the `confirmationState` as CONFIRM, which means that the user has confirmed the action, allowing it to be executed.

In [ ]:
raw_response_with_allowed = bedrock_agent_runtime_client.invoke_agent(
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    sessionState={
        'invocationId': event["returnControl"]["invocationId"],
        'returnControlInvocationResults': [{
                'apiResult': {
                    'actionGroup': event["returnControl"]["invocationInputs"][0]["apiInvocationInput"]["actionGroup"],
                    'apiPath': event["returnControl"]["invocationInputs"][0]["apiInvocationInput"]["apiPath"],
                    'confirmationState': 'CONFIRM',
                    'httpMethod': event["returnControl"]["invocationInputs"][0]["apiInvocationInput"]["httpMethod"],
                    'httpStatusCode': 200,
                    'responseBody': {
                        "TEXT": {
                            'body': ''
                        }
                    }
                }
        }]}
)

print(raw_response_with_allowed)

In [ ]:
%%time
event_stream = raw_response_with_allowed['completion']
for event_allowed in event_stream:
    print(event_allowed)

## Invoking an Agent with function results and execution is NOT allowed

As shown in the previous example, we are going to call the same method `invoke_agent`, but this time we will pass the option as `DENY`, which means the user did not permit the execution.

In [ ]:
raw_response_with_not_allowed = bedrock_agent_runtime_client.invoke_agent(
    agentId=agent_id,
    agentAliasId=agent_alias_id, 
    sessionId=session_id,
    enableTrace=enable_trace, 
    sessionState={
        'invocationId': event["returnControl"]["invocationId"],
        'returnControlInvocationResults': [{
                'apiResult': {
                    'actionGroup': event["returnControl"]["invocationInputs"][0]["apiInvocationInput"]["actionGroup"],
                    'apiPath': event["returnControl"]["invocationInputs"][0]["apiInvocationInput"]["apiPath"],
                    'confirmationState': 'DENY',
                    'httpMethod': event["returnControl"]["invocationInputs"][0]["apiInvocationInput"]["httpMethod"],
                    'httpStatusCode': 200,
                    'responseBody': {
                        "TEXT": {
                            'body': ''
                        }
                    }
                }
        }]}
)

print(raw_response_with_not_allowed)

In [ ]:
%%time
event_stream = raw_response_with_not_allowed['completion']
for event_not_allowed in event_stream:
    print(event_not_allowed)

After sending the `DENY` option in the `confirmationState`, you should receive the following message:

`Unfortunately, I was unable to send the reminder for claim claim-006 due to an access issue.`

### Clean up (optional)
The next steps are optional and demonstrate how to delete our agent. To delete the agent we need to:
1. update the action group to disable it
2. delete agent action group
3. delete agent alias
4. delete agent
5. delete lambda function
6. empty created s3 bucket
7. delete s3 bucket

In [ ]:
 # This is not needed, you can delete agent successfully after deleting alias only
# Additionaly, you need to disable it first

action_group_id = agent_action_group_response['agentActionGroup']['actionGroupId']
action_group_name = agent_action_group_response['agentActionGroup']['actionGroupName']

response = bedrock_agent_client.update_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id,
    actionGroupName=action_group_name,
    actionGroupExecutor={
        'lambda': lambda_function['FunctionArn']
    },
    apiSchema={
        's3': {
            's3BucketName': bucket_name,
            's3ObjectKey': bucket_key
        }
    },
    actionGroupState='DISABLED',
)

action_group_deletion = bedrock_agent_client.delete_agent_action_group(
    agentId=agent_id,
    agentVersion='DRAFT',
    actionGroupId= action_group_id
)

In [ ]:
agent_alias_deletion = bedrock_agent_client.delete_agent_alias(
    agentId=agent_id,
    agentAliasId=agent_alias['agentAlias']['agentAliasId']
)

In [ ]:
agent_deletion = bedrock_agent_client.delete_agent(
    agentId=agent_id
)

In [ ]:
# Delete Lambda function
lambda_client.delete_function(
    FunctionName=lambda_name
)

In [ ]:
# Empty and delete S3 Bucket

objects = s3_client.list_objects(Bucket=bucket_name)  
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key']) 
s3_client.delete_bucket(Bucket=bucket_name)

In [ ]:
# Delete IAM Roles and policies

for policy in [bedrock_agent_bedrock_allow_policy_name, bedrock_agent_s3_allow_policy_name]:
    iam_client.detach_role_policy(RoleName=agent_role_name, PolicyArn=f'arn:aws:iam::{account_id}:policy/{policy}')
    
iam_client.detach_role_policy(RoleName=lambda_role_name, PolicyArn='arn:aws:iam::aws:policy/service-role/AWSLambdaBasicExecutionRole')

for role_name in [agent_role_name, lambda_role_name]:
    iam_client.delete_role(
        RoleName=role_name
    )

for policy in [agent_bedrock_policy, agent_s3_schema_policy]:
    iam_client.delete_policy(
        PolicyArn=policy['Policy']['Arn']
)

## Happy coding!